In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable

from PIL import Image

In [30]:
test_image_path = 'D:\\knee\\data\\triathlon\\triathlon1.jpg'

In [3]:
%%time
rn50_model = models.resnet50(pretrained=True)
# model = model.cuda()
rn50_model.eval()
# layer = rn34_model._modules.get('avgpool')

Wall time: 2.36 s


In [4]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [5]:
class FeatureExtract(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.features = nn.Sequential(
            *list(base_model.children())[:-1]
        )
    def forward(self, x):
        x = self.features(x)
        return x

model = FeatureExtract(rn50_model)


In [6]:
def load_img(image_path):
    img = Image.open(image_path)
    t_img = Variable(normalize(to_tensor(scaler(img))))
#     print(t_img.shape)
    t_img = t_img.unsqueeze(0)
#     print(t_img.shape)
    return t_img

In [31]:
vector_1 = model(load_img(test_image_path))

In [8]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

In [9]:
def similarity(source_img_path, target_vec, sim_fn):
    t_img = load_img(source_img_path)
    if t_img.shape[1] != 3:
        return 0.0
    source_vec = model(t_img)
    return float(sim_fn(source_vec, target_vec)[0][0][0].data.cpu().numpy())

In [10]:
from pathlib import Path

In [ ]:
source_dir = Path('D:\\knee\\knee-xrays-master\\data\\raw\\output\\')

In [ ]:
%%time
for image_path in source_dir.iterdir():
    sim_val = similarity(image_path, target_vec=vector_1, sim_fn=cos)
    print(f'{image_path.stem},{sim_val}')

In [32]:
test_dir = Path('D:\\knee\\data_complete\\normalised\\output')

In [33]:
def find_top_k(test_dir):
    from collections import defaultdict
    
    sim_list = []
    
    for image_path in test_dir.iterdir():
        sim_val = similarity(image_path, target_vec=vector_1, sim_fn=cos)
        name = str(image_path.stem)
        sim_list.append((name, sim_val))
        #print(f'{image_path.stem},{sim_val}')
    return sim_list

In [34]:
sim_list = find_top_k(test_dir)

In [35]:
sim_list.sort(key=lambda tup: tup[1], reverse=True)  # sorts in place

In [36]:
#print (((test_image_path.split('/'))[:-1]).split('.'))
var = str(test_image_path)
listme = (var.split('\\'))[-1]
test_file_name = (listme.split('.'))[0]
test_name = test_file_name[:-1]
print (test_name)

triathlon


In [37]:
def calc_recall_precision(sim_list):
    truepositive = truenegative = falsepositive = falsenegative = 0
    for element in sim_list:
        if element[1] > 0.8:
            #print (str(element[0]))
            if test_name in str(element[0]).lower():
                truepositive += 1
            else:
                falsepositive += 1
        else:
            if test_name in str(element[0]).lower():
                falsenegative += 1
            else:
                truenegative += 1
    if((truepositive + falsepositive)!=0):
        precision = truepositive/(truepositive+falsepositive)
    else:
        precision = -1
    if((truepositive+falsenegative)!=0):
        recall = truepositive/(truepositive+falsenegative)
    else:
        recall = -1
    if(precision!= -1 and recall!= -1):
        F1 = (2*precision*recall)/(precision+recall)
    else:
        F1 = -1
    print (truepositive)
    print (truenegative)
    print (falsepositive)
    print (falsenegative)
    print (precision)
    print (recall)
    print (F1)


calc_recall_precision(sim_list)

6
41
72
1
0.07692307692307693
0.8571428571428571
0.1411764705882353


In [38]:
def num_correct_top_k(sim_list,k):
    correct_num = wrong_num = 0
    for element in sim_list[:k]:
        if test_name in str(element[0]).lower():
            correct_num +=1
        else:
            wrong_num +=1
    print (correct_num)
    print (wrong_num)
    
num_correct_top_k(sim_list,10)

3
7
